In [1]:
import csv
import re
import numpy as np
import pandas as pd
import nltk
import codecs
import collections as co
from collections import Counter
from collections import defaultdict
from openpyxl import load_workbook

In [2]:
df_reader = pd.read_csv(r"C:\Users\ysxia\Dropbox\Code\MutualFund\Data\2020_07\2020_07_10_MF_sample_all_columns_with_duplicates.csv")

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_reader.shape

(132343, 91)

In [4]:
df_reader.head(3)

,Unnamed: 0,Unnamed: 0.1,accession#,doc_filing,filing_year,fund_fam_name,fund_CIK,sec_act,SEC_fund_file#,SEC_film_#,...,strat_passe,strat_sent,risk_tense,risk_passe,risk_sent,crsp_class,strategy_word_counts,risk_word_counts,risk_word_counts_o,strategy_word_counts_o
0,0,0,0000004568-10-000004,497k,2010,american balanced fund,4568,1933 act,002-10758,10638475.0,...,0.166667,12,0.000000,0.000000,15,M,314,463,463.0,314.0
1,1,1,0000005138-10-000045,497k,2011,american growth fund inc,5138,1933 act,002-14543,101282152.0,...,0.000000,4,0.000000,0.000000,2,NaN,70,128,128.0,70.0
2,2,2,0000015260-10-000035,497k,2010,"midas perpetual portfolio, inc.",15260,1933 act,002-57953,10812274.0,...,0.333333,9,0.074074,0.070175,57,NaN,293,1107,1107.0,293.0


In [5]:
#checks for modified words around anchor words
def modifyWords(currentPosition, sentence, tempValue, finalModifyingDictionaries, specialModifyingList):

    windowCount = 0
    #creates a list of the five words before and after the anchor word[s]
    surroundingWords = []
    for currentNum in range(1,11):
        if currentPosition - 6 + currentNum >= 0 and currentNum <= 5:
            surroundingWords.append(sentence[currentPosition - 6 + currentNum].lower())
            windowCount +=1
        elif currentNum <= 5:
            surroundingWords.append(None)
        if len(sentence)-currentPosition >= currentNum-4 and currentNum > 5:
            surroundingWords.append(sentence[currentPosition + currentNum-4 - 1].lower())
            windowCount +=1
        elif currentNum > 5:
            surroundingWords.append(None)

    #Call helper functions
    modifyOnce = [False]* 10 #can only be modal, degree, or uncertain once, so this keeps track if the word has already been categorized
    for items in finalModifyingDictionaries:
        tempValue, modifyOnce = checkDictionary(specialModifyingList, items, modifyOnce, tempValue, surroundingWords)
    if windowCount != 0:
        return(tempValue/windowCount)
    else:
        return tempValue

In [6]:
#helper function to modifyingWords to check the dictionary
def checkDictionary(specialModifyingList, diction, modifyOnce, tempValue, surroundingWords): #checks if the word is in the dictionary
    modifyOnceCurrWord = 0 #need to see if the word is used for modifyOnce list
    for position in range(len(surroundingWords)):
        foundDoubleWord = False
        if not foundDoubleWord:
            words = surroundingWords[position]
            if words != None:
                try:
                    bothWords = words + " " + surroundingWords[position + 1]
                except:
                    bothWords = ""
                if not modifyOnce[modifyOnceCurrWord] and (bothWords.lower() in diction):
                    tempValue *= float(diction.get(bothWords.lower()))
                    modifyOnce[modifyOnceCurrWord]= True
                elif not modifyOnce[modifyOnceCurrWord] and (words.lower() in diction):
                    tempValue *= float(diction.get(words.lower()))
                    modifyOnce[modifyOnceCurrWord]= True
                for specialSubItems in specialModifyingList:
                    if 'unless' in specialSubItems[0] and words != "" and specialSubItems[0] in diction and words in specialSubItems[0] and (words != specialSubItems[0][:specialSubItems[0].find('unless')-1]):
                        tempValue *= float(specialSubItems[1])
                        modifyOnce[modifyOnceCurrWord]= True
                    if 'within' in specialSubItems[0] and specialSubItems[0] in diction and('within' != specialSubItems[0][:specialSubItems[0].find('within')]) and words == specialSubItems[0][:specialSubItems[0].find('within')-1]:
                        number = int(specialSubItems[0][specialSubItems[0].find('within')+6: specialSubItems[0].find('within')+8])
                        try:
                            if specialSubItems[0][specialSubItems[0].find('within')+9:] in ' '.join(surroundingWords[temp:temp + number + 2]):
                                tempValue *= float(specialSubItems[1])
                                modifyOnce[modifyOnceCurrWord]= True
                        except:
                            try:
                                if specialSubItems[0][specialSubItems[0].find('within')+9:] in ' '.join(surroundingWords[temp:temp + number + 1]):
                                    tempValue *= float(specialSubItems[1])
                                    modifyOnce[modifyOnceCurrWord]= True
                            except:
                                pass
                        try:
                            if specialSubItems[0][specialSubItems[0].find('within')+9:] in ' '.join(surroundingWords[temp - number - 2: temp]):
                                tempValue *= float(specialSubItems[1])
                                modifyOnce[modifyOnceCurrWord]= True
                        except:
                            try:
                                if specialSubItems[0][specialSubItems[0].find('within')+9:] in ' '.join(surroundingWords[temp - number - 1: temp]):
                                    tempValue *= float(specialSubItems[1])
                                    modifyOnce[modifyOnceCurrWord]= True
                            except:
                                pass
            modifyOnceCurrWord += 1
        else:
            foundDoubleWord = True
    print (tempValue, modifyOnce)
    return [tempValue, modifyOnce]

In [7]:
#FUNCTION THAT GOES THROUGH THE DICTIONARY AND FIGURES OUT THE WORD VALUE
#The way that it is currently written, the code has a disclosure value (nothing is averaged for the fund family or anything).
#It just generates the value for each sentence and puts it in a list and averages them together in the main function below
def lookupWord(sentence, modifyingDict, anchorDictionaries, specialAnchorList, specialModifyingList):
    if "a" in sentence:
        sentence.remove("a")
    if "i" in sentence:
        sentence.remove("i")
    if "o" in sentence:
        sentence.remove("o")
    if "A" in sentence:
        sentence.remove("A")
    if "I" in sentence:
        sentence.remove("I")
    if "O" in sentence:
        sentence.remove("O")
    anchorWords, sentenceValue, anchorWordList = 0, 0, []
    for currentPosition in range(len(sentence)):
        foundDoubleWord = False #need to keep track of words in dictionary with two words
        if not foundDoubleWord:
            wordInData = sentence[currentPosition]
            try:
                twoWords = wordInData + " " + sentence[currentPosition + 1]
            except:
                twoWords = ""
            keepLooking = True #if turns false, then means word is already categoried as negative, positive, or legal, can't be all three
            for ancDict in anchorDictionaries:
                if keepLooking:
                    sentenceValue, anchorWords, anchorWordList, keepLooking, foundDoubleWord = checkDictionaryAnchors(twoWords, specialAnchorList, specialModifyingList, modifyingDict, ancDict, wordInData, currentPosition, sentence, sentenceValue, anchorWords, anchorWordList)  
                    print (sentenceValue, anchorWords, anchorWordList, keepLooking)
                    
        else:
            foundDoubleWord = False
    return [sentenceValue, anchorWords, anchorWordList]

In [8]:
#Helper funciton to lookupWord to check the dictionary for anchorwords
def checkDictionaryAnchors(bothWords, specialAnchorList, specialModifyingList, modifyingDict, diction, wordInData, currentPosition, sentence, sentenceValue, anchorWords, anchorWordList):
    for specialSubItems in specialAnchorList: #if there are two words that need to be matched to dictionary checks them
        if 'unless' in specialSubItems[0] and bothWords != "" and bothWords in specialSubItems[0] and (bothWords != specialSubItems[0][:specialSubItems[0].find('unless')-1]):
            anchorWords += 1
            anchorWordList.append(bothWords)
            sentenceValue += modifyWords(currentPosition, sentence, specialSubItems[1], modifyingDict, specialModifyingList)
            return [sentenceValue, anchorWords, anchorWordList, False, False]
        if 'within' in specialSubItems[0] and ('within' != specialSubItems[0][:specialSubItems[0].find('within')]) and wordInData == specialSubItems[0][:specialSubItems[0].find('within')-1]:
            number = int(specialSubItems[0][specialSubItems[0].find('within')+6: specialSubItems[0].find('within')+8])
            try:
                if specialSubItems[0][specialSubItems[0].find('within')+9:] in ' '.join(sentence[temp:temp + number + 2]):
                    anchorWords += 1
                    anchorWordList.append(bothWords)
                    sentenceValue += modifyWords(currentPosition, sentence, specialSubItems[1], modifyingDict, specialModifyingList)
                    return [sentenceValue, anchorWords, anchorWordList, False, False]
            except:
                try:
                    if specialSubItems[0][specialSubItems[0].find('within')+9:] in ' '.join(sentence[temp:temp + number + 1]):
                        anchorWords += 1
                        anchorWordList.append(bothWords)
                        sentenceValue += modifyWords(currentPosition, sentence, specialSubItems[1], modifyingDict, specialModifyingList)
                        return [sentenceValue, anchorWords, anchorWordList, False, False]
                except:
                    pass
            try:
                if specialSubItems[0][specialSubItems[0].find('within')+9:] in ' '.join(sentence[temp - number - 2: temp]):
                    anchorWords += 1
                    anchorWordList.append(bothWords)
                    sentenceValue += modifyWords(currentPosition, sentence, specialSubItems[1], modifyingDict, specialModifyingList)
                    return [sentenceValue, anchorWords, anchorWordList, False, False]
            except:
                try:
                    if specialSubItems[0][specialSubItems[0].find('within')+9:] in ' '.join(sentence[temp - number - 1: temp]):
                        anchorWords += 1
                        anchorWordList.append(bothWords)
                        sentenceValue += modifyWords(currentPosition, sentence, specialSubItems[1], modifyingDict, specialModifyingList)
                        return [sentenceValue, anchorWords, anchorWordList, False, False]
                except:
                    pass
    if bothWords.lower() in diction: #makes dicitonary word lower case, and skips the header in the dictionary
        anchorWords += 1
        anchorWordList.append(bothWords)
        sentenceValue += modifyWords(currentPosition, sentence, diction.get(bothWords.lower()), modifyingDict, specialModifyingList)
        return [sentenceValue, anchorWords, anchorWordList, False, True]    
    elif wordInData.lower() in diction: #makes dicitonary word lower case, and skips the header in the dictionary
        anchorWords += 1
        anchorWordList.append(wordInData)
        sentenceValue += modifyWords(currentPosition, sentence, diction.get(wordInData.lower()), modifyingDict, specialModifyingList)
        return [sentenceValue, anchorWords, anchorWordList, False, False]
    else:
        return [sentenceValue, anchorWords, anchorWordList, True, False]

In [9]:
############################################################################
#code from stack overflow website to seperate sentences in a disclosure into a list
abbreviations = {'dr.': 'doctor', 'mr.': 'mister', 'bro.': 'brother', 'bro': 'brother', 'mrs.': 'mistress', 'ms.': 'miss', 'jr.': 'junior', 'sr.': 'senior',
                 'e.g.': 'for example', 'vs.': 'versus', 'U.S.': 'United States','etc.': 'etcetera', 'J.P.': 'Justice of the Peace',
                 'Inc.': 'Incorporated', 'LLC.': 'limited liability corporation', 'Co.': 'company', 'l.p.': 'limited partneship',
                 'ltd.': 'limited', 'Jan.': 'January', 'Feb.': 'February', 'Mar.': 'March', 'Apr.': 'April', 'i.e.': 'for example',
                 'Jun.': 'June', 'Jul.': 'July', 'Aug.': 'August', 'Oct.': 'October', 'Dec.': 'December', 'S.E.C.': 'SEC', 'Inv. Co. Act': 'Investment Company Act'}
terminators = ['.', '!', '?']
wrappers = ['"', "'", ')', ']', '}']

In [10]:
def find_sentence_end(paragraph):
    [possible_endings, contraction_locations] = [[], []]
    contractions = abbreviations.keys()
    sentence_terminators = terminators + [terminator + wrapper for wrapper in wrappers for terminator in terminators]
    for sentence_terminator in sentence_terminators:
        t_indices = list(find_all(paragraph, sentence_terminator))
        possible_endings.extend(([] if not len(t_indices) else [[i, len(sentence_terminator)] for i in t_indices]))
    for contraction in contractions:
        c_indices = list(find_all(paragraph, contraction))
        contraction_locations.extend(([] if not len(c_indices) else [i + len(contraction) for i in c_indices]))
    possible_endings = [pe for pe in possible_endings if pe[0] + pe[1] not in contraction_locations]
    if len(paragraph) in [pe[0] + pe[1] for pe in possible_endings]:
        max_end_start = max([pe[0] for pe in possible_endings])
        possible_endings = [pe for pe in possible_endings if pe[0] != max_end_start]
    possible_endings = [pe[0] + pe[1] for pe in possible_endings if sum(pe) > len(paragraph) or (sum(pe) < len(paragraph) and paragraph[sum(pe)] == ' ')]
    end = (-1 if not len(possible_endings) else max(possible_endings))
    return end


In [11]:
def find_sentences(paragraph):
   replacingList = [["?", "? "], ["!", "! "],[".", ". "],["(", " "], [")", " "], [",", " "], ["i. e.", "i.e"], ["e. g.", "e.g."],["U. S.", "U.S."], ["J. P.", "J.P."], ["l. p.", "l.p."], ["S. E. C.", "S.E.C."]]
   for items in replacingList:
       paragraph = paragraph.replace(items[0], items[1])
   paragraph = re.sub(' +', ' ', paragraph)
   if paragraph != "":
       if paragraph[0] == '?' or paragraph[0] == '.' or paragraph[0] == '!':
           paragraph = paragraph[1:]
   end = True
   sentences = []
   while end > -1:
       end = find_sentence_end(paragraph)
       if end > -1:
           sentences.append(paragraph[end:].strip())
           paragraph = paragraph[:end]
   sentences.append(paragraph)
   sentences.reverse()
   return sentences

In [12]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1:
            return
        yield start
        start += len(sub)

In [13]:
#create a list of the Dictionaries
#dictionary order: Negator, Modal, Degree, Uncertain
file_path = r"C:\Users\ysxia\Dropbox\Code\MutualFund\Sentiment working paper\2020_09\dictionary.xlsx"
negatorDict = pd.read_excel(file_path, sheet_name="TX_NEGATOR_CON", usecols = [0,1], encoding = 'utf-16')
modalDict = pd.read_excel(file_path, sheet_name="TX_MODAL_CON", skiprows = 0, usecols = [0,1], encoding = 'utf-16')
degreeDict = pd.read_excel(file_path, sheet_name="TX_DEGREE_CON", usecols = [0,1], encoding = 'utf-16')
uncertDict = pd.read_excel(file_path, sheet_name="TX_UNCERT_CON", usecols = [0,1], encoding = 'utf-16')
negativeDict = pd.read_excel(file_path, sheet_name="TX_NEGATIVE_CON", usecols = [0,1], encoding = 'utf-16')
positiveDict = pd.read_excel(file_path, sheet_name="TX_POSITIVE_ST_CON", usecols = [0,1], encoding = 'utf-16')
legalDict = pd.read_excel(file_path, sheet_name="TX_LEGAL_CON", usecols = [0,1], encoding = 'utf-16')



In [15]:
pd.set_option('display.max_rows', 10000)
with open('2020_12_senti_dictionary.txt', 'w') as f:
    print('negator list:\n', negatorDict, file=f)
    print('modalDict list:\n', modalDict,file=f)
    print('degreeDict list:\n', degreeDict, file=f)
    print('uncertDict list:\n', uncertDict, file=f)
    print('negativeDict list:\n', negativeDict, file=f)
    print('positiveDict list:\n', positiveDict, file=f)
    print('legalDict list:\n', legalDict, file=f)


In [16]:
#create the python modifying dictionaries
dictionaries = [negatorDict, modalDict, degreeDict, uncertDict]
finalModifyingDictionaries = []
specialModifyingList = []
for dictionary in dictionaries:
    emptyDict = {}
    for (key, value) in dictionary.values:
        if 'unless' in str(key).lower() and 'unless' != str(key).lower():
            specialModifyingList.append([str(key).lower(), int(value), 0])
        elif 'within' in str(key).lower() and 'within' != str(key).lower():
            specialModifyingList.append([str(key).lower(), int(value), 0])
        else:
            emptyDict[str(key).lower()] = float(value)
    finalModifyingDictionaries.append(emptyDict)


In [17]:
#create the python anchor dictionaries
dictionaries2 = [negativeDict, positiveDict, legalDict]
finalAnchorDictionaries = []
specialAnchorList = []
for dictionary in dictionaries2:
    emptyDict = {}
    for (key, value) in dictionary.values:
        if 'unless' in str(key).lower() and 'unless' != str(key).lower():
            specialAnchorList.append([str(key).lower(), int(value)])
        elif 'within' in str(key).lower() and 'within' != str(key).lower():
            specialAnchorList.append([str(key).lower(), int(value)])
        else:
            emptyDict[str(key).lower()] = float(value)
    finalAnchorDictionaries.append(emptyDict)

In [18]:
df_reader.head()

,Unnamed: 0,Unnamed: 0.1,accession#,doc_filing,filing_year,fund_fam_name,fund_CIK,sec_act,SEC_fund_file#,SEC_film_#,...,strat_passe,strat_sent,risk_tense,risk_passe,risk_sent,crsp_class,strategy_word_counts,risk_word_counts,risk_word_counts_o,strategy_word_counts_o
0,0,0,0000004568-10-000004,497k,2010,american balanced fund,4568,1933 act,002-10758,10638475.0,...,0.166667,12,0.000000,0.000000,15,M,314,463,463.0,314.0
1,1,1,0000005138-10-000045,497k,2011,american growth fund inc,5138,1933 act,002-14543,101282152.0,...,0.000000,4,0.000000,0.000000,2,NaN,70,128,128.0,70.0
2,2,2,0000015260-10-000035,497k,2010,"midas perpetual portfolio, inc.",15260,1933 act,002-57953,10812274.0,...,0.333333,9,0.074074,0.070175,57,NaN,293,1107,1107.0,293.0
3,3,3,0000032859-10-000005,497k,2010,endowments /de,32859,1933 act,002-34371,101098682.0,...,0.000000,9,0.000000,0.000000,5,DE,205,145,145.0,205.0
4,4,4,0000035315-10-000047,497k,2010,fidelity fixed income trust,35315,1933 act,002-41839,101150386.0,...,0.000000,8,0.000000,0.000000,16,FI,137,269,269.0,137.0


In [19]:
df_reader.iloc[10004].principal_strategies

"main investments. the fund invests at least 65% of its total assets in us and foreign equities (equities issued by us and foreign-based companies). although the fund can invest in companies of any size and from any country, it invests mainly in common stocks of established companies in countries with developed economies. the fund's equity investments may also include preferred stocks and other securities with equity characteristics, such as convertible securities and warrants. management process. in choosing securities, portfolio management uses a combination of three analytical disciplines: o bottom-up research. portfolio management looks for individual companies that it believes have a history of above-average growth, strong competitive positioning, attractive prices relative to potential growth, sound financial strength and effective management, among other factors. o growth orientation. portfolio management generally looks for companies that it believes have above-average potentia

In [20]:
df_reader['risk_sentiment']=""
df_reader['strategy_sentiment']=""
df_reader['strategy_plus_risk_sentiment']=""

In [21]:
specialAnchorList

[['assume unless within 1 example, chart, performance', -2],
 ['assumed unless within 1 example, chart, performance', -2],
 ['assumes unless within 1 example, chart, performance', -2],
 ['default unless credit default swaps, default swaps', -2],
 ['late unless in late', -2],
 ['less unless less developed', -2],
 ['limitation unless without limitation', -2],
 ['liquidate unless within 2 fund', -2],
 ['manipulation within 1 market', -2],
 ['manipulations within 1 market', -2],
 ['growth unless growth company', 2],
 ['outstanding unless outstanding loans', 2]]

In [22]:
specialModifyingList

[['contingencies unless 1 is, are, subject to', 0, 0],
 ['contingent unless 1 is, are, subject to', 0, 0]]

In [19]:
# find the strategy section sentiment score
position = 0
for row in range(len(df_reader)): #goes through each row in the spreadsheet with the mutual fund/investment disclosures
   # paragraph = row[7]###############THIS LINE OF THE CODE IS REFERRING TO GRABBING THE FOURTH COLUMN, OR THE RISK SECTION. CHANGE THIS IF APPLYING SCORE TO DIFFERENT COLUMN/INVESTMENT SECTION
    paragraph = df_reader.iloc[row].principal_strategies
    paragraphVals = []
    #rowWriting = row[0:9]
    for items in find_sentences(paragraph): #find the sentences in the paragraph
        try:
            if items[-1] == '.' or items[-1] == '!' or items[-1] == '?':
                items = items[:-1]
            items = items.replace(';', '')
        except:
            pass
        wordList = items.split() #for each sentence create a list of the words in the sentence
        sentenceValAnswers = lookupWord(wordList, finalModifyingDictionaries, finalAnchorDictionaries, specialAnchorList, specialModifyingList) #and figure out the value for the sentence by passing it to lookupWord function
        sentenceVal, anchorWords, anchorList = sentenceValAnswers[0], sentenceValAnswers[1], sentenceValAnswers[2]
        if sentenceVal != 0 and anchorWords != 0:
            #print (sentenceVal, anchorWords, sentenceVal/anchorWords)
            paragraphVals.append(sentenceVal/anchorWords) #adds the value returned to a list so that we can see the values for each sentence in a paragraph
        else:                                
            paragraphVals.append(sentenceVal) #adds the value returned to a list so that we can see the values for each sentence in a paragraph
    
    df_reader['strategy_sentiment'].loc[row] = np.mean(paragraphVals)

    position+=1
    if position == 2000 or position == 20000 or position == 50000 or position == 70000:
        print(position)
    
    
    #rowWriting.append(np.mean(paragraphVals))
    
    #writer.writerow(rowWriting)
#common.close()
#csvfile.close()

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


2000
20000
50000
70000


In [20]:
df_reader.head(10)

,Unnamed: 0,Unnamed: 0.1,accession#,doc_filing,filing_year,fund_fam_name,fund_CIK,sec_act,SEC_fund_file#,SEC_film_#,...,risk_passe,risk_sent,crsp_class,strategy_word_counts,risk_word_counts,risk_word_counts_o,strategy_word_counts_o,risk_sentiment,strategy_sentiment,strategy_plus_risk_sentiment
0,0,0,0000004568-10-000004,497k,2010,american balanced fund,4568,1933 act,002-10758,10638475.0,...,0.000000,15,M,314,463,463.0,314.0,,0.0865327,
1,1,1,0000005138-10-000045,497k,2011,american growth fund inc,5138,1933 act,002-14543,101282152.0,...,0.000000,2,NaN,70,128,128.0,70.0,,0.065997,
2,2,2,0000015260-10-000035,497k,2010,"midas perpetual portfolio, inc.",15260,1933 act,002-57953,10812274.0,...,0.070175,57,NaN,293,1107,1107.0,293.0,,-0.0347222,
3,3,3,0000032859-10-000005,497k,2010,endowments /de,32859,1933 act,002-34371,101098682.0,...,0.000000,5,DE,205,145,145.0,205.0,,0.078817,
4,4,4,0000035315-10-000047,497k,2010,fidelity fixed income trust,35315,1933 act,002-41839,101150386.0,...,0.000000,16,FI,137,269,269.0,137.0,,0.0162187,
5,5,5,0000035315-10-000048,497k,2010,fidelity fixed income trust,35315,1933 act,002-41839,101150388.0,...,0.000000,18,others,155,299,299.0,155.0,,0.0283807,
6,6,6,0000035315-10-000049,497k,2010,fidelity fixed income trust,35315,1933 act,002-41839,101150391.0,...,0.123457,81,others,149,1960,1960.0,149.0,,0.0283807,
7,7,7,0000035315-10-000050,497k,2010,fidelity fixed income trust,35315,1933 act,002-41839,101150392.0,...,0.000000,18,others,156,299,299.0,156.0,,0.0283807,
8,8,8,0000035315-10-000051,497k,2010,fidelity fixed income trust,35315,1933 act,002-41839,101150395.0,...,0.000000,18,Index,116,290,290.0,116.0,,-0.0114286,
9,9,9,0000035315-10-000052,497k,2010,fidelity summer street trust,225322,1933 act,002-58542,101150400.0,...,0.138462,65,DE,93,1537,1537.0,93.0,,0.01,


In [21]:
#find risk section sentiment score
position = 0
for row in range(len(df_reader)): #goes through each row in the spreadsheet with the mutual fund/investment disclosures
   # paragraph = row[7]###############THIS LINE OF THE CODE IS REFERRING TO GRABBING THE FOURTH COLUMN, OR THE RISK SECTION. CHANGE THIS IF APPLYING SCORE TO DIFFERENT COLUMN/INVESTMENT SECTION
    paragraph = df_reader.iloc[row].principal_risks
    paragraphVals = []
    #rowWriting = row[0:9]
    for items in find_sentences(paragraph): #find the sentences in the paragraph
        try:
            if items[-1] == '.' or items[-1] == '!' or items[-1] == '?':
                items = items[:-1]
            items = items.replace(';', '')
        except:
            pass
        wordList = items.split() #for each sentence create a list of the words in the sentence
        sentenceValAnswers = lookupWord(wordList, finalModifyingDictionaries, finalAnchorDictionaries, specialAnchorList, specialModifyingList) #and figure out the value for the sentence by passing it to lookupWord function
        sentenceVal, anchorWords, anchorList = sentenceValAnswers[0], sentenceValAnswers[1], sentenceValAnswers[2]
        if sentenceVal != 0 and anchorWords != 0:
            #print (sentenceVal, anchorWords, sentenceVal/anchorWords)
            paragraphVals.append(sentenceVal/anchorWords) #adds the value returned to a list so that we can see the values for each sentence in a paragraph
        else:                                
            paragraphVals.append(sentenceVal) #adds the value returned to a list so that we can see the values for each sentence in a paragraph
    
    df_reader['risk_sentiment'].loc[row] = np.mean(paragraphVals)

    position+=1
    if position == 2000 or position == 20000 or position == 50000 or position == 70000:
        print(position)
    
    


2000
20000
50000
70000


In [25]:
df_reader.iloc[1].principal_strategies 

'investment research corporation (the "adviser") manages the fund using a growth style of investing. we use a consistent approach to build the fund´s security portfolio which is made up primarily of common stocks and securities convertible into common stock. these securities are issued by large companies, and to a lesser extent, small and mid-sized companies. when a company´s fundamentals are strong, we believe earnings growth will follow.'

In [26]:
df_reader.iloc[1].principal_risks

'of investing in the fund the primary risks of investing in the fund are: ~ stock market risk - the value of an investment may fluctuate, ~ industry and security risk - risks relating to an industry as a whole or a company´s prospects for business success, ~ management risk - risks that the adviser´s assessment of a company´s growth prospects may not be accurate, ~ liquidity risk - a given security or asset may not be readily marketable, ~ small cap risk - small cap stock risk tend to have a high exposure to market fluctuations and failure, ~ mid cap risk - mid cap stock risk tend to have a greater exposure to market fluctuations and failure. loss of some or all of the money you invest is a risk of investing in the fund.'

In [29]:
 df_reader.iloc[1].principal_risks + "  "+ df_reader.iloc[1].principal_strategies 

'of investing in the fund the primary risks of investing in the fund are: ~ stock market risk - the value of an investment may fluctuate, ~ industry and security risk - risks relating to an industry as a whole or a company´s prospects for business success, ~ management risk - risks that the adviser´s assessment of a company´s growth prospects may not be accurate, ~ liquidity risk - a given security or asset may not be readily marketable, ~ small cap risk - small cap stock risk tend to have a high exposure to market fluctuations and failure, ~ mid cap risk - mid cap stock risk tend to have a greater exposure to market fluctuations and failure. loss of some or all of the money you invest is a risk of investing in the fund.  investment research corporation (the "adviser") manages the fund using a growth style of investing. we use a consistent approach to build the fund´s security portfolio which is made up primarily of common stocks and securities convertible into common stock. these secu

In [30]:
#find strategy plus risk section sentiment score
position = 0
for row in range(len(df_reader)): #goes through each row in the spreadsheet with the mutual fund/investment disclosures
   # paragraph = row[7]###############THIS LINE OF THE CODE IS REFERRING TO GRABBING THE FOURTH COLUMN, OR THE RISK SECTION. CHANGE THIS IF APPLYING SCORE TO DIFFERENT COLUMN/INVESTMENT SECTION
    paragraph = df_reader.iloc[row].principal_strategies + " " + df_reader.iloc[row].principal_risks
    paragraphVals = []
    #rowWriting = row[0:9]
    for items in find_sentences(paragraph): #find the sentences in the paragraph
        try:
            if items[-1] == '.' or items[-1] == '!' or items[-1] == '?':
                items = items[:-1]
            items = items.replace(';', '')
        except:
            pass
        wordList = items.split() #for each sentence create a list of the words in the sentence
        sentenceValAnswers = lookupWord(wordList, finalModifyingDictionaries, finalAnchorDictionaries, specialAnchorList, specialModifyingList) #and figure out the value for the sentence by passing it to lookupWord function
        sentenceVal, anchorWords, anchorList = sentenceValAnswers[0], sentenceValAnswers[1], sentenceValAnswers[2]
        if sentenceVal != 0 and anchorWords != 0:
            #print (sentenceVal, anchorWords, sentenceVal/anchorWords)
            paragraphVals.append(sentenceVal/anchorWords) #adds the value returned to a list so that we can see the values for each sentence in a paragraph
        else:                                
            paragraphVals.append(sentenceVal) #adds the value returned to a list so that we can see the values for each sentence in a paragraph
    
    df_reader['strategy_plus_risk_sentiment'].loc[row] = np.mean(paragraphVals)

    position+=1
    if position == 2000 or position == 20000 or position == 50000 or position == 70000:
        print(position)
    
    

2000
20000
50000
70000


In [31]:
df_reader.shape

(132343, 94)

In [32]:
df_reader.head()

,Unnamed: 0,Unnamed: 0.1,accession#,doc_filing,filing_year,fund_fam_name,fund_CIK,sec_act,SEC_fund_file#,SEC_film_#,...,risk_passe,risk_sent,crsp_class,strategy_word_counts,risk_word_counts,risk_word_counts_o,strategy_word_counts_o,risk_sentiment,strategy_sentiment,strategy_plus_risk_sentiment
0,0,0,0000004568-10-000004,497k,2010,american balanced fund,4568,1933 act,002-10758,10638475.0,...,0.000000,15,M,314,463,463.0,314.0,-0.0381935,0.0865327,0.017733
1,1,1,0000005138-10-000045,497k,2011,american growth fund inc,5138,1933 act,002-14543,101282152.0,...,0.000000,2,NaN,70,128,128.0,70.0,-0.116477,0.065997,-0.0027781
2,2,2,0000015260-10-000035,497k,2010,"midas perpetual portfolio, inc.",15260,1933 act,002-57953,10812274.0,...,0.070175,57,NaN,293,1107,1107.0,293.0,-0.0717011,-0.0347222,-0.0625349
3,3,3,0000032859-10-000005,497k,2010,endowments /de,32859,1933 act,002-34371,101098682.0,...,0.000000,5,DE,205,145,145.0,205.0,-0.0883164,0.078817,0.0172181
4,4,4,0000035315-10-000047,497k,2010,fidelity fixed income trust,35315,1933 act,002-41839,101150386.0,...,0.000000,16,FI,137,269,269.0,137.0,-0.10965,0.0162187,-0.0713485


In [33]:
df_reader.to_csv("2020_08_20_MF_all_colunms_with_duplicats_with_sentiment_scores.csv")

In [36]:
df_reader.columns.values

array(['Unnamed: 0', 'Unnamed: 0.1', 'accession#', 'doc_filing',
       'filing_year', 'fund_fam_name', 'fund_CIK', 'sec_act',
       'SEC_fund_file#', 'SEC_film_#', 'former_name-1', 'former_name_2',
       'former_name_3', 'stock_series#', 'fund_name', 'class_#',
       'fund_class_ticker_1', 'fund_class_ticker_2',
       'fund_class_ticker_3', 'fund_class_ticker_4',
       'fund_class_ticker_5', 'fund_class_ticker_6',
       'fund_class_ticker_7', 'fund_class_ticker_8',
       'fund_class_ticker_9', 'class_type', 'class_#_2', 'class_type_2',
       'class_#_3', 'class_type_3', 'class_#_4', 'class_type_4',
       'class_#_5', 'class_type_5', 'class_#_6', 'class_type_6',
       'class_#_7', 'class_type_7', 'class_#_8', 'class_type_8',
       'class_#_9', 'class_type_9', 'principal_strategies',
       'principal_risks', 'total_word_counts', 'crsp_obj', 'filing_date',
       'risks_priceVolRisk', 'risks_priceVolRisk1', 'risks_priceVolRisk2',
       'risks_priceVolRisk3', 'risks_priceVolR

In [37]:
df_reader.shape

(132343, 94)

In [39]:
df_reader['filing_year'].value_counts()

2017    17203
2016    16610
2018    16357
2015    16169
2013    15287
2014    15052
2012    13226
2011    12933
2010     9489
2019       17
Name: filing_year, dtype: int64

In [40]:
df1 = df_reader[df_reader['filing_year']!=2019]

In [41]:
df1.shape

(132326, 94)

In [42]:
df1_dup = df1.sort_values('filing_year').drop_duplicates(subset=['filing_year','stock_series#'],keep='last')

In [43]:
df1_dup.shape

(77544, 94)

In [44]:
df1_dup.head()

,Unnamed: 0,Unnamed: 0.1,accession#,doc_filing,filing_year,fund_fam_name,fund_CIK,sec_act,SEC_fund_file#,SEC_film_#,...,risk_passe,risk_sent,crsp_class,strategy_word_counts,risk_word_counts,risk_word_counts_o,strategy_word_counts_o,risk_sentiment,strategy_sentiment,strategy_plus_risk_sentiment
0,0,0,0000004568-10-000004,497k,2010,american balanced fund,4568,1933 act,002-10758,10638475.0,...,0.000000,15,M,314,463,463.0,314.0,-0.0381935,0.0865327,0.017733
6360,6360,6360,0001193125-10-068753,497k,2010,massmutual premier funds,927972,1933 act,033-82366,10708442.0,...,0.045455,22,Index,401,544,544.0,401.0,-0.0998448,0.0363542,-0.0451254
6361,6361,6361,0001193125-10-068764,497k,2010,massmutual premier funds,927972,1933 act,033-82366,10708467.0,...,0.068966,29,DE,408,779,779.0,408.0,-0.0966418,0.0231647,-0.0501169
6362,6362,6362,0001193125-10-068772,497k,2010,massmutual premier funds,927972,1933 act,033-82366,10708503.0,...,0.000000,11,others,150,356,356.0,150.0,-0.0261922,0.0571429,-0.00629385
6363,6363,6363,0001193125-10-068775,497k,2010,massmutual premier funds,927972,1933 act,033-82366,10708506.0,...,0.068966,29,NaN,553,811,811.0,553.0,-0.0964209,-0.0057398,-0.0616539


In [45]:
df1.to_csv("2020_08_21_master_file_all_columns_with_duplicates.csv")
df1_dup.to_csv("2020_08_21_master_file_all_columns_no_duplicates.csv")

In [46]:
fund =pd.read_csv(r"C:\Users\ysxia\Dropbox\Code\MutualFund\Law_paper\August 2020\2020_07_10_sample_fund_allocation.csv")

In [47]:
fund_sentiment = pd.merge(fund, df1_dup, on=['accession#'], how='left')

In [48]:
fund.shape

(281225, 85)

In [49]:
fund_sentiment.shape

(281225, 178)

In [53]:
fund_sentiment.columns.values

array(['Unnamed: 0_x', 'Unnamed: 0.1_x', 'Unnamed: 0.1.1',
       'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'accession#',
       'fund_CIK_x', 'stock_series#_x', 'contract_cik', 'crsp_fundno',
       'filing_year_x', 'filing_date_x', 'fund_name_x', 'top_3_perc_p',
       'top_3_perc_f', 'security_num_p', 'security_num_f', 'stdev_p',
       'stdev_f', 'alloc_date_p', 'alloc_date_f', 'mtna_p', 'mtna_f',
       'annual_ret_p', 'annual_ret_f', 'mret_p', 'mret_f', 'mret_date_p',
       'mret_date_f', 'top_3_perc_p_o', 'top_3_perc_f_o',
       'security_num_p_o', 'security_num_f_o', 'stdev_p_o', 'stdev_f_o',
       'mret_p_o', 'mret_f_o', 'mtna_p_o', 'mtna_f_o', 'annual_ret_p_o',
       'annual_ret_f_o', 'crsp_class_x', 'risk_fog_x', 'strategy_fog_x',
       'risk_fog_top', 'strategy_fog_top', 'risks_derivCount_x',
       'strat_derivCount_x', 'MS_Rating_monthly', 'MS_Rating_yearly',
       'strategies_wc', 'risks_wc', 'log_strategies_wc', 'log_risks_wc',
       'MS_Rating_monthly_int', 'mor

In [55]:
fund_sentiment_inner = pd.merge(fund, df1_dup[['accession#','risk_sentiment', 'strategy_sentiment',
       'strategy_plus_risk_sentiment']], on=['accession#'], how='inner')

In [56]:
fund_sentiment_inner.shape

(166029, 88)

In [57]:
fund_sentiment_inner.to_csv("2020_08_21_fund_allocation_with_sentiment.csv")